# IMPLEMENTATION OF EXTERIOR POINT ALGORITHM IN PYTHON

In [1]:
import time
st = time.time()

#!!    input the  file in line 88   !!


def mps_to_txt_matrixes(file_name):
    import numpy as np
    f = open(file_name+'.mps') 
    
    x=[]
    EQin=[]
    MinMax= -1
    for i in f:
        splitted= i.split()
        x.append(splitted)
        flag =0
    for i in range(len(x)):
        if x[i][0]== 'ROWS' and flag == 0 :
            flag += 1
            row=i #where "ROWS"
        elif  x[i][0] == 'COLUMNS' and flag == 1:
            flag += 1
            colum= i #where "COLUMNS"
        elif x[i][0] == 'RHS' and flag == 2:
            flag += 1
            rhs = i #where rhs starts
        elif 'ENDATA' in x[i][0]:
            flag=4 
            end= i 
    # find the name of the objective function and append the EQin vector
    for i in range(2,colum):
        if x[i][0]== 'L':
            EQin.append(-1)
        elif x[i][0]== 'G':
            EQin.append(1)
        elif x[i][0]== 'E':
            EQin.append(0)
        else :
            name_objective= x[i][1]
        
    constraints=[]
    for i in range(row+2,colum):
        if x[i][1] not in constraints:
            constraints.append(x[i][1])
        
    variables=[]
    for i in range(colum+1,rhs):
        if (x[i][0]) not in variables:
            variables.append(x[i][0])
            
    #number of constraints
    num_constraints = len(constraints)
    
    #number of variables
    num_variables= len(variables)
    
    #create A zero matrix mxn(m= constraints and n = variables)
    A= np.zeros((num_constraints, num_variables))
    #create empty c array
    c = np.zeros((num_variables,), dtype=float)
    for i in range(colum+1, rhs):
        a= int(x[i][0][1:])
        for y in range(1,len(x[i]),2):
            #print(x[i][y])
            if x[i][y]== name_objective:
                c[a-1]+= float(x[i][y+1])
            elif y==1 or y==3:
                b= int(x[i][y][1:])
                A[b-1,a-1] +=  float(x[i][y+1])
        
        
    # b (1xm) 
    b = np.zeros((num_constraints,),dtype=float)  

    for i in range(rhs+1,end):
        for y in range(len(x[i])):
            if y==1 or y==3 :
                a = int(x[i][y][1:])
                
                b[a-1] +=float(x[i][y+1])
    
    return A, b ,c ,EQin
import numpy as np





A,b,c, Eqin =mps_to_txt_matrixes('sdata1_500x500')

m=len(A)
n=len(A[0])


print('number of constraints: ',m)
print('number of variables: ',n)


Eqin= np.transpose(Eqin)

nnz=np.count_nonzero(Eqin) #number of nonzeros in Eqin
w, h = len(A) , nnz        #dimensios of matrix with the new variables
bd = np.zeros((m, nnz)) 
bcols=0

for i in range(m):        #check inequalities, where eqin -1 replace with +1 to the bd matrix. Where 1 replace with -1.
    if Eqin[i] ==-1:
        bd[i,bcols] += 1
        bcols +=1
    elif Eqin[i] ==1:
        bd[i,bcols] -= 1
        bcols +=1
A_new = np.concatenate((A,bd), axis=1)     # New A ( A + bd) 
bd1=np.zeros(nnz)
c_new= np.concatenate((c,bd1), axis=0)

m= len(A_new) 
n=len(A_new[0])

## Zero step
B = np.array([]) #τικ
N = np.array([]) #τικ
P = np.array([]) 
Q = np.array([])
L = np.array([])
S0 = np.array([])
dB = np.array([])
Sp = np.array([])
eis1 = np.array([])
eis2 = np.array([])
Sq = np.array([])
hj = np.array([])
Wt = np.array([])
SnP = np.array([])


# N,B
for i in range(0,n-m):
    N= np.append(N,int(i))     
for i in range(n-m,n):
    B=np.append(B,int(i))

#A ---> An, Ab
An=np.zeros((m,len(N)))
Ab=np.zeros((m,len(B)))
x=0
y=0
Cn=[]
Cb=[]
x=0
for i in B:
    
    Ab[:,x]=A_new[:,int(i)]
    Cb.append(c_new[int(i)])
    x+=1

x=0
for i in N:
    An[:,x]=A_new[:,int(i)]
    Cn.append(c_new[int(i)])
    x+=1


Ab_inv=np.linalg.inv(Ab)


#Compute  Xb , Wt, Sn 
Xb= np.dot(Ab_inv,b.T)

Wt= np.dot(Cb,Ab_inv)

Sn= Cn - np.dot(Wt,An)



# N ---> P,Q 
for i in range(len(Sn)):
    if Sn[i] <=0 :
        P=np.append(P,N[i])
        
    else :
        Q=np.append(Q,N[i])
        


#create λ 
λ= np.zeros(len(P))+1


#create Ap , Aq 
Ap= np.zeros((m,len(P)))
Aq=np.zeros((m,len(Q)))
x=0
y=0
for i in N:
    if i in P:
        Ap[:,x]= A_new[:,int(i)]
        
        x+=1
    else:
        Aq[:,y]= A_new[:,int(i)]
        y+=1

# Compute S0 and db
S0=0
x=0
for i in range(len(Sn)):
    if Sn[i] <0:
        S0+= λ[x]*Sn[i]
        x+=1

db= np.zeros((m,1))-1
h=np.zeros(m)
for i in P:
    h += np.dot(Ab_inv,A[:,int(i)])
for i in range(len(db)):
    db[i]= db[i]*h[i]

#Step 1 
#compute α and β

β= float('-inf') #cause Xb >0 

x=[]
y=[]
for i in range(len(B)):
    if db[i]<0:
        x.append(-float(Xb[i]/db[i]))
    else:
        x.append(0)
        y.append(i)
        
α=min([x[i] for i in range(len(x))  if i not in y ])

r=x.index(α)

k=int(B[r])



#step 2 input variable selection
#compute Hrp and Hrq
Hrp= np.dot(Ab_inv[r,:],Ap)
Hrq=np.dot(Ab_inv[r,:],Aq)


#compute θ1 and θ2
th1= []
y=[]
for i in range(len(P)):
    if Hrp[i]>0:
        th1.append(-float((Sn[int(P[i])])/Hrp[i])) 
    else :
        th1.append(0)
        y.append(i)
#Sn is always negative for i in P so we want to find the min for only the positive numbers
try:
    θ1=min([th1[i] for i in range(len(th1)) if i not in  y])
except ValueError:
    θ1= float('inf')
#print('θ1:',θ1)

th2=[]
y=[]
for i in range(len(Q)):
    if Hrq[i]<0:
        th2.append(-float((Sn[int(Q[i])])/Hrq[i])) 
    else:
        th2.append(0)
        y.append(i)
#Sn is always positie  for i in Q so we want to find the min for only the positive numbers (-Positive/negative= positive)
try:
    θ2=min([th2[i] for i in  range(len(th2)) if i not in  y])
except ValueError:
    θ2= float('inf')
#print('θ2:',θ2)

#select the input variable l
if θ1<=θ2:
    l1=th1.index(θ1)
    l=int(P[l1])
else:
    l1=th2.index(θ2)
    l=int(Q[l1])

    
#1st iteration
#UPDATE PFI SCHEME
#Compute new  Ab inverse
E= np.identity(len(Ab))
hl=np.dot(Ab_inv,A_new[:,l])
hl_r= -hl/hl[r]
hl_r[r]=1/hl[r]
E[:,r]=hl_r
Ab_inv= np.dot(E,Ab_inv)
db= np.dot(E,db)



#step 3 Rotation 
if θ1 <= θ2:
    db[r]+=1
    P=np.array([int(i) for i in P if i != l])
    #print('new P:',P)
    Q=np.append(Q,k).astype(int)
    #print('new Q:',Q)
else:
    Q=np.array([int(i) for i in Q if i != l])
    Q=np.append(Q,k).astype(int)

N=np.concatenate((P,Q))


B[r]=l


#A ---> An, Ab

x=0
y=0

for i in B:
    Ab[:,x]=A_new[:,int(i)]
    Cb[x]=c_new[int(i)]
    x+=1

x=0
for i in N:
    An[:,x]=A_new[:,int(i)]
    Cn[x]=c_new[int(i)]
    x+=1


#Compute new  Xb , Wt, Sn 

Xb= np.dot(Ab_inv,b.T)

Wt= np.dot(Cb,Ab_inv)

Sn= Cn - np.dot(Wt,An)


#compute S0 
S0=0
x=[]
for i in range(len(P)): #now P elements are the first elements of N and then there are all the Q elements
    S0 += Sn[i]
    


Ap= np.zeros((m,len(P)))
Aq=np.zeros((m,len(Q)))
x=0
y=0
for i in N:
    if i in P:
        Ap[:,x]= A_new[:,int(i)]
    
        x+=1
    else:
        
        Aq[:,y]= A_new[:,int(i)]
        y+=1

#iter 2 and so on 

iretation=2 
while len(P)>0 and  S0!=0 and (all(val>0 for val in db)==False):
    #print('ieration:',iretation)
    #Step 1 
    #compute α and β
    β= float('-inf') #cause Xb >0 
   
    x=[]
    y=[]
    for i in range(len(B)):
        if db[i]<0:
            x.append(-float(Xb[i]/db[i]))
        else:
            x.append(0)
            y.append(i)
        
            
    α=min([x[i] for i in range(len(x))  if i not in y ])

    r=x.index(α)

    k=int(B[r])

    #'the output variable is : X'+str(k))



    #step 2 input variable selection
    #compute Hrp and Hrq
    Hrp= np.dot(Ab_inv[r,:],Ap)
    Hrq=np.dot(Ab_inv[r,:],Aq)
    

    #compute θ1 and θ2
    th1= []
    y=[]
    for i in range(len(P)):
        if Hrp[i]>0:
            th1.append(-float(Sn[i]/Hrp[i]))
        else:
            th1.append(0)
            y.append(i)
    
    try:
        θ1=min([th1[i] for i in range(len(th1)) if i not in  y])
    except ValueError:
        θ1= float('inf')
    
    th2=[]
    y=[]
    for i in range(len(P),len(N)):
        j=i-len(P)
        if Hrq[j]<0:
            th2.append(-float(Sn[i]/Hrq[j]))
        else:
            th2.append(0)
            y.append(j)
            
    
    try:
        θ2=min([th2[i] for i in  range(len(th2)) if i not in  y])
    except ValueError:
        θ2= float('inf')
    
    
    #select the input variable l
    if θ1<=θ2:
        
        l1=th1.index(θ1)
        l=int(P[l1])
        
    else:
        
        l1=th2.index(θ2)
        l=int(Q[l1])
        
            
    #
    #('Input variable is : X'+str(l))
     #UPDATE PFI SCHEME
    E= np.identity(len(Ab))
    hl=np.dot(Ab_inv,A_new[:,l])
    hl_r= -hl/hl[r]
    hl_r[r]=1/hl[r]
    E[:,r]=hl_r
    Ab_inv= np.dot(E,Ab_inv)
    db= np.dot(E,db)
    
    #step 3 Rotation 
    if θ1 <= θ2:
        db[r]+=1
        P=np.array([int(i) for i in P if i != l])
        
        Q=np.append(Q,k).astype(int)
        
    else:
        #('P is the same')
        Q=np.array([int(i) for i in Q if i != l])
        Q=np.append(Q,k).astype(int)
        
        
    N=np.concatenate((P,Q))
    
        
    B[r]=l

                    
    #A ---> An, Ab
    
    x=0
    y=0
    
    for i in B:
        Ab[:,x]=A_new[:,int(i)]
        Cb[x]=c_new[int(i)]
        x+=1
   
    x=0
    for i in N:
        An[:,x]=A_new[:,int(i)]
        Cn[x]=c_new[int(i)]
        x+=1
    


    

    
    
    #Compute new  Xb , Wt, Sn 
    Xb= np.dot(Ab_inv,b)
    
    Wt= np.dot(Cb,Ab_inv)
    
    Sn= Cn - np.dot(Wt,An)
  
    
    #compute S0 
    S0=0
    x=[]
    for i in range(len(P)): #now P elements are the first elements of N and then there are all the Q elements
        S0 += Sn[i]
        
   
    
    Ap= np.zeros((m,len(P)))
    Aq=np.zeros((m,len(Q)))
    x=0
    y=0
    for i in N:
        if i in P:
            Ap[:,x]= A_new[:,int(i)]    
            x+=1
        else:
            
            Aq[:,y]= A_new[:,int(i)]
            y+=1
   
    iretation+=1
    
print('After '+str(iretation)+' iretations the solution is :')
z= np.dot(Cb,Xb) #the optimal solution
print('z=',z)
et = time.time()
elapsed_time = et - st
print('Execution time:', elapsed_time, 'seconds')

number of constraints:  500
number of variables:  500
After 440 iretations the solution is :
z= -80.52772721479063
Execution time: 30.45630955696106 seconds
